In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate

In [2]:
# Function to load images and masks from a folder

def load_data(data_folder, labels_folder):
    images = []
    masks = []
    
    for filename in os.listdir(data_folder):
        if filename.endswith(".png") or filename.endswith(".jpg"):  # Adjust file extensions as needed
            file_path = os.path.join(data_folder, filename)
            
            # Load and resize the image
            image = cv2.imread(file_path)
            image = cv2.resize(image, (480, 640))  # Resize to match your model input size
            images.append(image)
            
            # Load and resize the corresponding mask
#             mask_filename = filename.replace(".png", "_mask.png")  # Assuming mask filenames follow a pattern
            mask_path = os.path.join(labels_folder, filename)
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        
            
            # Check if the mask is not empty before resizing
            if mask is not None and not mask.size == 0:
                mask = cv2.resize(mask, (480, 640))  # Resize to match your model input size
                masks.append(mask)
            else:
                print(f"Warning: Mask is empty for {filename}")
                       

    return np.array(images), np.array(masks)

# Load data
train_data_folder = "C:\\Users\\ashis\\OneDrive\\Desktop\\New folder\\data_Grayscale\\train_actual"
test_data_folder = "C:\\Users\\ashis\\OneDrive\\Desktop\\New folder\\data_Grayscale\\test"
val_data_folder = "C:\\Users\\ashis\\OneDrive\\Desktop\\New folder\\data_Grayscale\\validate"

train_labels_folder = "C:\\Users\\ashis\\OneDrive\\Desktop\\New folder\\data_label\\train_actual_label"
test_labels_folder = "C:\\Users\\ashis\\OneDrive\\Desktop\\New folder\\data_label\\test_label"
val_labels_folder = "C:\\Users\\ashis\\OneDrive\\Desktop\\New folder\\data_label\\validate_label"
# test_labels_folder = "C:\\Users\\ashis\\OneDrive\\Desktop\\New folder\\binary_label\\test_label"

train_images, train_masks = load_data(train_data_folder, train_labels_folder)
test_images, test_masks = load_data(test_data_folder, test_labels_folder)
val_images, val_masks = load_data(val_data_folder, val_labels_folder)


In [3]:
# Function to define U-Net model
def unet_model(input_size=(640, 480, 3)):
    inputs = Input(input_size)
    
    # Encoder
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    # Decoder
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    up1 = UpSampling2D(size=(2, 2))(conv2)
    
    # Output layer
    output = Conv2D(1, 1, activation='sigmoid')(up1)
    
    model = Model(inputs=inputs, outputs=output)
    return model


# Define and compile the U-Net model
model = unet_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



In [ ]:
# Train the model
model.fit(train_images, train_masks, validation_data=(val_images, val_masks), epochs=5, batch_size=32)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_images, test_masks)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

Epoch 1/5


In [ ]:
# Save the trained model
model.save('unet_model.h5')

try: 1

In [ ]:
# Test the model on a new video
# Assuming 'video_path' is the path to your video file
video_path = "C:\\path\\to\\your\\video.mp4"
cap = cv2.VideoCapture(video_path)

# Define a function to apply the trained U-Net model to each frame of the video
def apply_unet(frame):
    frame_resized = cv2.resize(frame, (640, 480))  # Resize to match the model input size
    frame_input = np.expand_dims(frame_resized, axis=0)
    predicted_mask = model.predict(frame_input)
    return predicted_mask[0]

# Process each frame of the video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Apply the U-Net model to get the predicted mask
    predicted_mask = apply_unet(frame)
    
    # Display the original frame and the predicted mask
    cv2.imshow("Original Frame", frame)
    cv2.imshow("Predicted Mask", predicted_mask)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
